In [3]:
import pandas as pd
df=pd.read_csv('../input/movielens/ratings.csv')
print(df)

In [4]:
print(df.nunique(axis = 0))

In [5]:
import seaborn as sns
sns.barplot(x=df.columns, y=df.nunique(axis=0), palette="Blues_d")

In [6]:
# univariate analysis
import matplotlib.pyplot as plt

plt.figure(1, figsize = (16,4))
#top 20 movies
df['movieId'].value_counts()[:20].plot(kind = 'bar') 
plt.figure(2, figsize = (16,4))
#top 20 users
df['userId'].value_counts()[:20].plot(kind = 'bar') 
plt.figure(3, figsize = (8,4))
df['rating'].plot(kind = 'hist')

In [7]:
ratings_per_user = df.groupby('userId')['movieId'].count()
ratings_per_user.hist()

In [8]:
ratings_per_movie = df.groupby('movieId')['userId'].count()
ratings_per_movie.hist()

In [9]:
import surprise
from surprise import SVD
# SVD
SVD_algo = SVD()

In [10]:
from surprise import Dataset
from surprise import Reader
# load dataframe into Surprise Reader object
reader = Reader(rating_scale = (0,5))
rating_dataframe = Dataset.load_from_df(df[['userId','movieId','rating']], reader)

In [13]:
from surprise.model_selection import train_test_split
from surprise import accuracy
# define train test function
def train_test_algo(algo, label):
    training_set, testing_set = train_test_split(rating_dataframe, test_size = 0.25)
    algo.fit(training_set)
    test_output = algo.test(testing_set)
    test_dataframe = pd.DataFrame(test_output)
    
    return test_dataframe

In [14]:
train_test_SVD = train_test_algo(SVD_algo, "SVD_algo")
print(train_test_SVD.head())

In [15]:
movie_dataframe = pd.read_csv("../input/movielens/movies.csv")
links_dataframe = pd.read_csv("../input/movielens/links.csv")

In [16]:
def prediction(algo, users_K):
    prediction_list = []
    for userId in range(1,users_K):
        for movieId in range(1,53889):
            rating = algo.predict(userId, movieId).est
            prediction_list.append([userId, movieId, rating])
    prediction_dataframe = pd.DataFrame(prediction_list, columns = ['userId', 'movieId', 'rating'])
    return prediction_dataframe

In [17]:
def top_recommendations_movies(prediction_dataframe, top_N):
    link_movie = pd.merge(prediction_dataframe, links_dataframe, how='inner', left_on='movieId', right_on='movieId')
    rec_movie = pd.merge(link_movie, movie_dataframe, how='left', left_on='movieId', right_on='movieId')[['userId', 'movieId', 'rating','imdbId','title']]
    sorted_df = rec_movie.groupby(('userId'), as_index = False).apply(lambda x: x.sort_values(['rating'], ascending = False)).reset_index(drop=True)
    top_rec_movies = sorted_df.groupby('userId').head(top_N)
    return sorted_df, top_rec_movies

In [18]:
## SVD predictions for the MovieLens Large Dataset
pred_SVD = prediction(SVD_algo, 11)
rec_movies_SVD, top_rec_movies_SVD = top_recommendations_movies(pred_SVD, 3)
print('\n')
print('\n')
print("Top Three Movies Recommendations for first 10 users")
print(top_rec_movies_SVD)